In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService
from src.utils.type_utils import create_langchain_doc

DATA_REPO = "BroDeadlines/TEST.TDT.mini.tdt_copora_data"
DATA_REPO = "BroDeadlines/TEST.UEH.ueh_copora_data"
SUBSET = "compact"
provider = ProviderService()

# Push Hugging Face dataset to ES

- data from hugging face
- push vector data 


In [2]:
dataset = load_dataset(DATA_REPO, split=SPLIT)
index = SPLIT.lower()

vec_idx = f"vec-{index}"
text_idx = f"text-{index}"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

In [3]:
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits'],
    num_rows: 344
})

In [4]:
from time import time
start = time()
a = [212]*455
end = time()
print(end - start)

4.863739013671875e-05


In [5]:
start = time()
for row in dataset:
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=split, metadata=meta) for split in row['splits']]
    es.add_documents(docs)
    bm.add_documents(docs)

end = time()

In [6]:
(end-start) / 60

5.594338897864024

In [8]:
vec_idx

'vec-index.medium_index_tdt'

In [9]:
text_idx

'text-index.medium_index_tdt'

# Push Proposition dataset to ES

In [2]:
proposition_ds = load_dataset(DATA_REPO)
proposition_ds

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'doc_id', 'metadata', 'split', 'shards', 'propositions', 'proposition_list', '__index_level_0__'],
        num_rows: 196
    })
})

In [5]:
from collections import Counter
lengths = []
list_lengths = []

def check_length(row):
    # lnstr = f"len{len(row['proposition_str'])}"
    list_lengths.append(f"len{len(row['proposition_list'])}")
    # lengths.append(lnstr)
    return

proposition_ds.map(check_length)
print(Counter(lengths))
# Counter(list_lengths)

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Counter()


In [3]:
# proposition_filtered_ds = proposition_ds.filter(lambda e: e['url'] == "https://tuvanhocduong.tdtu.edu.vn/News" and sum([len(a) for a in e['propositions']]) > 0)
proposition_filtered_ds = proposition_ds.filter(lambda e: len(e['proposition_list']) > 0)
proposition_filtered_ds

Filter:   0%|          | 0/196 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'doc_id', 'metadata', 'split', 'shards', 'propositions', 'proposition_list', '__index_level_0__'],
        num_rows: 195
    })
})

## Full-text option

In [4]:
index = "ueh-unique"

vec_idx = f"vec-sentence-{index}"
text_idx = f"text-sentence-{index}"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

errors = []

def build_doc_upload(row):
    # full-text
    propos = row['propositions']
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=prop, metadata=meta) for prop in row['propositions']]
    try:
        es.add_documents(docs)
        bm.add_documents(docs)
    except:
        errors.append(meta['doc_id'])
        print("ERROR")
    return

def build_doc_upload_v2(row):
    # sentence-level
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=prop, metadata=meta) for prop in row['proposition_list']]
    try:
        es.add_documents(docs)
        bm.add_documents(docs)
    except:
        errors.append(meta['doc_id'])
        print("ERROR")
    return

In [5]:
print(vec_idx)
print(text_idx)

vec-sentence-ueh-unique
text-sentence-ueh-unique


In [6]:
proposition_filtered_ds.map(build_doc_upload_v2)

Parameter 'function'=<function build_doc_upload_v2 at 0x7f80347e35e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/195 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'content', 'doc_id', 'metadata', 'split', 'shards', 'propositions', 'proposition_list', '__index_level_0__'],
        num_rows: 195
    })
})

In [11]:
errors

[]